# Workflow

There are two workflows for NMX, one for simulation and one for measurement data. <br>
This page will show simulation data workflow as an example. <br>
They are written with ``sciline``, so we will show how to collect ``providers`` and ``parameters`` to build a workflow pipeline and compute the required result. <br>

First, we will set up scipp logging widget in the notebook.

In [ ]:
# Logging widget
from scipp.logging import get_log_widget, make_widget_handler, get_logger, WidgetHandler

sc_logger = get_logger()
if not any([hdlr for hdlr in sc_logger.handlers if isinstance(hdlr, WidgetHandler)]):
    sc_logger.addHandler(make_widget_handler())
    sc_logger.setLevel('DEBUG')
    sc_logger.info('NMX Data reduction - part 1.')

get_log_widget()

## TL;DR

If you don't need to replace any providers or parameters, you can use the pre-defined pipeline.

In [ ]:
import sciline as sl
from ess.nmx.workflow import collect_default_parameters, providers
from ess.nmx.loader import FileTypeMcStas, InputFileName, MaximumPropability, DefaultMaximumPropability
from ess.nmx.data import small_mcstas_sample
from ess.nmx.reduction import TimeBinned, TimeBinStep

file_path = small_mcstas_sample()  # Replace it with your data file path

nmx_workflow = sl.Pipeline(list(providers), params={
                            **collect_default_parameters(),
                            MaximumPropability: DefaultMaximumPropability,
                            TimeBinStep: TimeBinStep(1),
                            InputFileName: InputFileName(file_path),
                        })

time_binned = nmx_workflow.compute(TimeBinned[FileTypeMcStas])
time_binned

Pretreatment to make it faster <br>
not needed with new McStas versions

```bash
h5repack -l CHUNK=1024x6 2e11.h5 2e11-rechunk.h5
# or
h5repack -l CHUNK=NONE 2e11.h5 2e11-nochunk.h5
# or
h5repack -l CHUNK=1024x6 mccode.h5 mccode-nochunk.h5 
```

Test data filename must be the exact file to use for this step of data reduction.

## Collect Providers and Parameters

There are helpers to collect providers and default parameters in ``ess.nmx.workflow`` module. <br>
If you need to replace any parameters, you can replace them in the dictionary and use it to build a pipeline. <br>
``build_workflow`` helper also accepts certain types as an argument to build a pipeline with, but not all types are supported by this helper. 

In [ ]:
import scipp as sc
from ess.nmx.workflow import collect_default_parameters, providers
from ess.nmx.loader import InputFileName

# Collect Parameters
# ``TimeBinStep`` and ``InputFileName`` are not included in the default parameters.
params = {
    **collect_default_parameters(),
    TimeBinStep: TimeBinStep(1),
    InputFileName: small_mcstas_sample()
}

# Parameters to run the workflow:
sc.DataGroup({param_t.__name__: val for param_t, val in params.items()})


## Build Pipeline

In [ ]:
import sciline as sl
from ess.nmx.loader import FileTypeMcStas
from ess.nmx.reduction import TimeBinned

pl = sl.Pipeline(
    list(providers),
    params=params
)

file_type = FileTypeMcStas
mcstas_workflow_graph = pl.get(TimeBinned[FileTypeMcStas])
mcstas_workflow_diagram = mcstas_workflow_graph.visualize(graph_attr={'rankdir': 'LR'})
mcstas_workflow_diagram.render('mcstas_workflow_graph', 'png')
mcstas_workflow_diagram

## Compute

You can compute a certain type from the graph.

In [ ]:
time_binned = mcstas_workflow_graph.compute(TimeBinned[FileTypeMcStas])
time_binned

You can also compute more than 1 type at the same time. <br>
Then ``compute`` returns a dictionary of requested types.

In [ ]:
from ess.nmx.reduction import GroupedByPixelID

multiple_results = pl.get((GroupedByPixelID[file_type], TimeBinned[file_type])).compute()
grouped = multiple_results[GroupedByPixelID[file_type]]

grouped